## 11주

11주 은행
은행 컨트랙을 프로그램하여 아래 기능을 실행하도록 하세요.

* 입금액을 매개변수로 넣어서 입금하세요. amount가 실제 입금액과 일치하지 않으면 오류.

	function deposit(uint amount)

* 전액 인출 함수를 구현하세요. 인출 금액을 정하지 않아도 되므로 함수의 인자가 없다.

	function widthdrawAll()

* 컨트랙 잔고확인 함수 (this를 이용한 잔고, 상태변수 잔고 (this 잔고가 맞는지 확인하는 용도)

	function getBalance() public view returns(uint, uint)

* 다른 계정으로 계좌이체 함수

	function forwardTo(address payable _receiver) public payable

* fallback()을 입금가능하도록 구현하세요. fallback이 호출되면 이벤트가 발생하여 전송자와 금액을 출력하세요.

이벤트 함수명은 PrintLog(address from, uint amount)



주피터 노트북에서 컴파일, 배포한 후, 아래 기능을 실행하여 그 결과를 출력하세요.

ganache 8335 띄우고, 노드로 하세요.

- 입금 11111 wei, 222 wei를 하고 컨트랙잔고 11333 wei 출력

- 자신의 2번째 계정으로 계좌이체 333 wei하고 컨트랙잔고 11000 wei 출력, 자신의 2번째 계정 잔고 증가분 (+333 wei) 출력

- 전액 인출하고 11000 wei, 자신의 잔고 증가분 출력 (+11000 wei)

- 컨트랙으로 111 송금하고, deposit()이 아니라 fallback으로 입금하세요. 컨트랙잔고 111 wei 출력.

In [1]:
%%writefile src/Bank.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract Bank {
    address owner;
    mapping (address => uint) balanceOf;
    constructor() {
        owner = msg.sender;
    }
    event PrintLog(address from, uint amount);
    
    function deposit(uint amount) public payable {
        require(msg.value == amount);
        balanceOf[owner] += amount;
    }
    
    function withdrawAll() public {
        //balanceOf[owner] -= address(this).balance;
        payable(owner).transfer(address(this).balance);
    }
    
    function getBalance() public view returns(uint, uint) {
        return (address(this).balance, balanceOf[owner]);
    }
    
    function forwardTo(address receiver, uint amount) public payable {
        balanceOf[owner] -= amount; 
        balanceOf[receiver] += amount;  
        require(msg.sender == owner);
        payable(receiver).transfer(msg.value);
        emit PrintLog(receiver, msg.value); 
    }
    
    fallback() external {
        emit PrintLog(owner, address(this).balance);
    }
}

Overwriting src/Bank.sol


In [2]:
!solc --optimize --combined-json abi,bin src/Bank.sol > src/Bank.json

In [3]:
%%writefile src/BankDeploy.js
var Web3 = require('web3');
var _abiBinJson = require('./Bank.json');  
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts);
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
_bin=_abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 364124}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/BankDeploy.js


In [4]:
!node src/BankDeploy.js

- contract name:  [ 'src/Bank.sol:Bank' ]
Deploying the contract from 0x51a74dbccbf2036d79341F8c151f495F61310080
hash: 0x9741498772b80987c55ad8ac88faee4266d5fae248c8c67331547b6bb3ad5c34
---> The contract deployed to: 0xE5610f1de37d8499920E37D528cd8885f87B9480


In [5]:
%%writefile src/BankUse.js
var Web3 = require('web3');
var _abiBinJson = require('./Bank.json');  
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));


contractName=Object.keys(_abiBinJson.contracts);
_abiArray=_abiBinJson.contracts[contractName].abi;
console.log("- ABI: " + _abiArray);
var bank = new web3.eth.Contract(_abiArray,"0xE5610f1de37d8499920E37D528cd8885f87B9480");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    const balance0Before = await web3.eth.getBalance(accounts[0]);
    console.log("Balance0 before: " + balance0Before);
    const balance1Before = await web3.eth.getBalance(accounts[1]);
    console.log("Balance1 before: " + balance1Before);
    bank.methods.getBalance().call().then(function(bal) {
        console.log("Contract bal before first deposit: " + bal[0]);
    });
    await bank.methods.deposit(11111).send({from: accounts[0], value:11111});
    bank.methods.getBalance().call().then(function(bal) {
        console.log("Contract bal after first deposit: " + bal[0]);
    });
    bank.methods.getBalance().call().then(function(bal) {
        console.log("Contract bal before second deposit: " + bal[0]);
    });
    await bank.methods.deposit(222).send({from: accounts[0], value:222});
    bank.methods.getBalance().call().then(function(bal) {
        console.log("Contract bal after second deposit: " + bal[0]);
    });
    const forward = await bank.methods.forwardTo(accounts[1], 333).send({from: accounts[0], value:333});
    bank.methods.getBalance().call().then(function(bal) {
        console.log("Contract balance after forwardTo: " + bal[1]);
    });
    const balance1After = await web3.eth.getBalance(accounts[1]);
    console.log("Balance1 after: " + balance1After);
    const withdraw = await bank.methods.withdrawAll();
    bank.methods.getBalance().call().then(function(bal) {
        console.log("Contract balance after withdrawAll: " + bal[1]);
    });

}

doIt()

Overwriting src/BankUse.js


In [6]:
!node src/BankUse.js

- ABI: [object Object],[object Object],[object Object],[object Object],[object Object],[object Object],[object Object]
Balance0 before: 999999444021999988889
Balance1 before: 1000000000000000000000
Contract bal before first deposit: 0
Contract bal after first deposit: 11111
Contract bal before second deposit: 11111
Contract bal after second deposit: 11333
Balance1 after: 1000000000000000000333
Contract balance after forwardTo: 11000
Contract balance after withdrawAll: 11000
